In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available ibmqx2 instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the ibmqx2 instead
device = provider.get_backend('ibmqx2')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 80
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    conditional: False
    sample_name: sparrow
    quantum_volume: 8
    memory: True
    max_experiments: 75
    online_date: 2017-01-24T05:00:00+00:00
    open_pulse: False
    description: 5 qubit device
    allow_q_object: True
    allow_object_storage: True
    backend_name: ibmqx2
    allow_q_circuit: False
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    credits_required: True
    max_shots: 8192
    url: None
    n_registers: 1

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 50.31548 µs / 53.02446 µs / 0.0 / 0.00074 / 0.00147 / 0.019
    Q1 / 5.2379 GHz / 73.73208 

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
transQcList = []

pi = math.pi

In [6]:
import numpy as np
def input_state(circ, nqubits, value):
    for j in range(nqubits):
        circ.h(j)
        circ.u1(-value * math.pi/float(2**(j)), j)
def qft(circ, q, n):
    """n-qubit QFT on q in circ."""
    for j in range(n):
        for k in range(j):
            circ.cu1(np.pi / float(2**(j - k)), q[j], q[k])
        circ.h(q[j])

In [7]:
def qftCircuit_classical():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    
    for i in range(4):
        circuit.h(q[i])
    
    qft(circuit, q, 4)

    for k in range(4):
        circuit.measure(q[k],c[k])
    
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots
   
def qftCircuit_classical_assertion():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    for i in range(4):
        circuit.h(q[i])

    qft(circuit, q, 4)

    qubitList = [q[3]]
    success = classical_assertion(circuit, qubitList, 0)
    if not success:
        print("Assertion circuit error")

    for k in range(4):
        circuit.measure(q[k],c[k])
        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [8]:
def qftExperimentClassical():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    num_cnots_original = -1
    for i in range(10):
        trans_qc, num_cnots = qftCircuit_classical()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qftCircuit_classical()
        if num_cnots == min_cnots:
            print("\nnumber of CNOTs in circuit without assertion: ", num_cnots)
            num_cnots_original = num_cnots
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = qftCircuit_classical_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qftCircuit_classical_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 15:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break

In [9]:
def qftCircuit_superposition():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)
    
    circuit.x(q[2])
    #swap 1 and 3
    for j in range(4):
        m = j
        if j == 1:
            m = 3
        if j == 3:
            m = 1
        for k in range(j):
            n = k
            if k == 1:
                n = 3
            if k == 3:
                n = 1
                
            circuit.cu1(np.pi / float(2**(j - k)), q[m], q[n])
        circuit.h(q[m])


    circuit.u1(-pi/4, q[0])
    circuit.h(q[0])
    circuit.u1(-pi/2, q[3])
    circuit.h(q[3])
    circuit.u1(-pi, q[2])
    circuit.h(q[2])
    circuit.h(q[1])
    circuit.measure(q[0],c[0])
    circuit.measure(q[3],c[1])
    circuit.measure(q[2],c[2])
    circuit.measure(q[1],c[3])
    
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

def qftCircuit_superposition_assertion():
    q = QuantumRegister(4)
    c = ClassicalRegister(4)
    circuit = QuantumCircuit(q,c)

    circuit.x(q[2])
    #swap 1 and 3
    for j in range(4):
        m = j
        if j == 1:
            m = 3
        if j == 3:
            m = 1
        for k in range(j):
            n = k
            if k == 1:
                n = 3
            if k == 3:
                n = 1
                
            circuit.cu1(np.pi / float(2**(j - k)), q[m], q[n])
        circuit.h(q[m])

    circuit.u1(-pi/4, q[0])
    circuit.h(q[0])

    circuit.u1(-pi, q[2])
    circuit.h(q[2])
    circuit.h(q[1])
    
    #assert for q[3]
    qubitList = [q[3]]
    phaseDict = {q[3]:[pi/2, pi/2]}
    success = superposition_assertion(circuit, qubitList, phaseDict, flag = 1)
    if not success:
        print("Assertion circuit error")
    #invert the phase so the output state is |0000>
    circuit.u1(-pi/2, q[3])
    circuit.h(q[3])
    
    circuit.measure(q[0],c[0])
    circuit.measure(q[3],c[1])
    circuit.measure(q[2],c[2])
    circuit.measure(q[1],c[3])

        
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [10]:
def qftExperimentSuperposition():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    num_cnots_original = -1
    for i in range(10):
        trans_qc, num_cnots = qftCircuit_superposition()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qftCircuit_superposition()
        if num_cnots == min_cnots:
            print("\nnumber of CNOTs in circuit without assertion: ", num_cnots)
            num_cnots_original = num_cnots
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = qftCircuit_superposition_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = qftCircuit_superposition_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 15:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break

In [11]:
qftExperimentClassical()
qftExperimentSuperposition()


number of CNOTs in circuit without assertion:  16
number of CNOTs in circuit with assertion:  17
    Percentage of the assertion circuit size in total size:  5.882352941176472 %

number of CNOTs in circuit without assertion:  17
number of CNOTs in circuit with assertion:  20
    Percentage of the assertion circuit size in total size:  15.000000000000002 %


In [12]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of QFT without classical assertion:

In [13]:
calcSuccessrate(res.get_counts(0), ["0000"], 0)

total_count =  8192 success_count =  2584 success_rate =  31.54296875 %


Success rate of QFT with classical assertion:

In [14]:
calcSuccessrate(res.get_counts(1), ["0 0000"], 1)

total_count =  5974 success_count =  4535 success_rate =  75.91228657515903 %


Success rate of QFT without superposition assertion:

In [15]:
calcSuccessrate(res.get_counts(2), ["0000"], 0)

total_count =  8192 success_count =  4403 success_rate =  53.74755859375 %


Success rate of QFT with superposition assertion

In [16]:
calcSuccessrate(res.get_counts(3), ["0 0000"], 1)

total_count =  5564 success_count =  2747 success_rate =  49.37095614665708 %
